In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
def coord(image):
    image = cv2.imread(image,cv2.IMREAD_UNCHANGED)
    # Ensure image has an alpha channel
    if image.shape[2] < 4:
        raise ValueError("Image does not have an alpha channel")

    # Extract the alpha channel
    alpha = image[:, :, 3]

    # Find all non-transparent pixels
    coords = cv2.findNonZero(alpha)  # Returns coordinates of non-zero (non-transparent) pixels

    # Get bounding box for non-transparent region
    x, y, w, h = cv2.boundingRect(coords)

    return x,y,w,h

In [10]:
troops = ["archer"]
data = pd.read_csv("image_centers/troops_center.csv")

for troop in troops:
    print(troop)
    image = f'images/troops/Blue/{troop}_run_n_6.png'
    x,y,w,h = coord(image)
    new_row = pd.DataFrame({'troop': [troop], 'x': [x], "y" : [y], "w": [w], "h" : [h]})
    data = pd.concat([data, new_row], ignore_index=True)
data.to_csv("image_centers/troops_center.csv",index=False)

archer


In [12]:
cv2.circle(image, (x,y), 2, (255,0,0), 2)
image_rgb =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'


In [7]:
data = pd.read_csv("/Users/tusharsingharoy/coderoyale/data/image_centers/troops_center.csv",index_col = 'troop')
data.loc['giant']

x     78
y    145
w    114
h     98
Name: giant, dtype: int64

In [3]:
from PIL import Image, ImageFilter

# Load the image
image_path = "/Users/tusharsingharoy/coderoyale/data/images/troops/Blue/archer_attack_e_4.png"
image = Image.open(image_path).convert("RGBA")  # Preserve transparency

# Get the bounding box (to determine the bottom axis)
bbox = image.getbbox()
if not bbox:
    raise ValueError("No bounding box found. Ensure the image is not fully transparent.")

# Create a black shadow based on the entire image (no cropping)
shadow = Image.new("RGBA", image.size, (0, 0, 0, 255))  # Fully black shadow
shadow_mask = image.split()[3]  # Use the alpha channel of the original image

# Stretch the shadow downward (project it on the ground)
shadow_width = image.width
shadow_height = int(image.height * 0.5)  # Adjust how far the shadow extends
shadow = shadow.resize((shadow_width, shadow_height))
shadow_mask = shadow_mask.resize((shadow_width, shadow_height))  # Resize the mask to match the shadow

# Apply blur to soften the shadow effect
shadow = shadow.filter(ImageFilter.GaussianBlur(10))

# Create a new blank image to accommodate both the original image and the shadow
new_width = image.width
new_height = image.height + shadow.height  # Extra space below for shadow
new_image = Image.new("RGBA", (new_width, new_height), (255, 255, 255, 0))

# Positioning
image_x = 0
image_y = 0  # Keep the original image in its position
shadow_x = 0  # Align shadow with image
shadow_y = image.height - 100  # Slight overlap to blend smoothly

# Paste the shadow first
new_image.paste(shadow, (shadow_x, shadow_y), mask=shadow_mask)

# Paste the original image on top
new_image.paste(image, (image_x, image_y), mask=image)

# Save the output image
output_path = "/Users/tusharsingharoy/coderoyale/data/shadow.png"
new_image.save(output_path)


print(f"Black shadow added without cropping. Image saved to {output_path}")


Black shadow added without cropping. Image saved to /Users/tusharsingharoy/coderoyale/data/shadow.png
